In [1]:
# https://docs.cohere.com/docs/embed-on-langchain
# https://www.pinecone.io/learn/refine-with-rerank/

# pip install langchain-cohere
# pip install cohere

from langchain.retrievers import ContextualCompressionRetriever
from langchain_cohere import CohereRagRetriever
from langchain_cohere import CohereRerank
from langchain_cohere import CohereEmbeddings
from langchain_cohere import ChatCohere
from langchain.storage import InMemoryStore
from langchain.retrievers import ParentDocumentRetriever
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma

import os
import openai
from dotenv import load_dotenv

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
app_dir = os.path.join(parent_dir, "")
load_dotenv(os.path.join(app_dir, ".env"))
openai_api_key = os.getenv("OPENAI_API_KEY")
cohere_api_ky = os.getenv("COHERE_API_KEY")
embed_model = "text-embedding-3-small"

In [2]:
raw_documents = TextLoader('dados/lei9279.txt', encoding='utf-8').load()

In [3]:
text_splitter = CharacterTextSplitter(chunk_size=512, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)
documents[0].page_content

Created a chunk of size 704, which is longer than the specified 512
Created a chunk of size 684, which is longer than the specified 512
Created a chunk of size 695, which is longer than the specified 512
Created a chunk of size 545, which is longer than the specified 512
Created a chunk of size 671, which is longer than the specified 512
Created a chunk of size 601, which is longer than the specified 512
Created a chunk of size 727, which is longer than the specified 512
Created a chunk of size 535, which is longer than the specified 512
Created a chunk of size 723, which is longer than the specified 512
Created a chunk of size 553, which is longer than the specified 512
Created a chunk of size 626, which is longer than the specified 512
Created a chunk of size 1172, which is longer than the specified 512
Created a chunk of size 610, which is longer than the specified 512
Created a chunk of size 518, which is longer than the specified 512
Created a chunk of size 730, which is longer th

'LEI Nº 9.279, DE 14 DE MAIO DE 1996\n\nRegula direitos e obrigações relativos à propriedade industrial.\n\n        O  PRESIDENTE DA REPÚBLICA Faço saber que o Congresso Nacional decreta e eu sanciono a seguinte Lei:\n\nDISPOSIÇÕES PRELIMINARES\n\n        Art. 1º Esta Lei regula direitos e obrigações relativos à propriedade industrial.'

In [4]:
documents[1].page_content

'Art. 2º A proteção dos direitos relativos à propriedade industrial, considerado o seu interesse social e o desenvolvimento tecnológico e econômico do País, efetua-se mediante:\n\n        I - concessão de patentes de invenção e de modelo de utilidade;\n\n        II - concessão de registro de desenho industrial;\n\n        III - concessão de registro de marca;\n\n        IV - repressão às falsas indicações geográficas; e\n\n        V - repressão à concorrência desleal.'

In [5]:
len(documents)

259

In [6]:
cohere_embeddings = CohereEmbeddings(cohere_api_key=os.getenv("COHERE_API_KEY"),model="embed-english-v3.0")
db = Chroma.from_documents(documents, cohere_embeddings)

# teste se tem algum db ativo
input_docs = db.as_retriever().invoke("tem alguma base ativa?")
try:
    print(input_docs[0])
except Exception as e:
    print(e)

page_content='I - aos atos praticados por terceiros não autorizados, em caráter privado e sem finalidade comercial, desde que não acarretem prejuízo ao interesse econômico do titular da patente;

        II - aos atos praticados por terceiros não autorizados, com finalidade experimental, relacionados a estudos ou pesquisas científicas ou tecnológicas;' metadata={'source': 'dados/lei9279.txt'}


In [7]:
db.delete(where={"source": "dados/bitcoin.txt"})
db.delete(where={"source": "dados/lei9279.txt"})

In [8]:
cohere_embeddings = CohereEmbeddings(cohere_api_key=os.getenv("COHERE_API_KEY"),model="embed-english-v3.0")
db = Chroma.from_documents(documents, cohere_embeddings)

In [9]:
user_query = "O que é um modelo de utilidade ?"
input_docs = db.as_retriever().get_relevant_documents(user_query)

C:\Users\otimi\AppData\Local\Temp\ipykernel_13804\3856611624.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  input_docs = db.as_retriever().get_relevant_documents(user_query)


In [10]:
input_docs

[Document(metadata={'source': 'dados/lei9279.txt'}, page_content='Art. 8º É patenteável a invenção que atenda aos requisitos de novidade, atividade inventiva e aplicação industrial.\n#\n        Art. 9º É patenteável como modelo de utilidade o objeto de uso prático, ou parte deste, suscetível de aplicação industrial, que apresente nova forma ou disposição, envolvendo ato inventivo, que resulte em melhoria funcional no seu uso ou em sua fabricação.\n#\n        Art. 10. Não se considera invenção nem modelo de utilidade:'),
 Document(metadata={'source': 'dados/lei9279.txt'}, page_content='Art. 88. A invenção e o modelo de utilidade pertencem exclusivamente ao empregador quando decorrerem de contrato de trabalho cuja execução ocorra no Brasil e que tenha por objeto a pesquisa ou a atividade inventiva, ou resulte esta da natureza dos serviços para os quais foi o empregado contratado.      (Regulamento)\n\n        § 1º Salvo expressa disposição contratual em contrário, a retribuição pelo trab

In [11]:
# https://docs.cohere.com/docs/embed-on-langchain

input_docs = db.as_retriever().invoke(user_query)
input_docs[0]

Document(metadata={'source': 'dados/lei9279.txt'}, page_content='Art. 8º É patenteável a invenção que atenda aos requisitos de novidade, atividade inventiva e aplicação industrial.\n#\n        Art. 9º É patenteável como modelo de utilidade o objeto de uso prático, ou parte deste, suscetível de aplicação industrial, que apresente nova forma ou disposição, envolvendo ato inventivo, que resulte em melhoria funcional no seu uso ou em sua fabricação.\n#\n        Art. 10. Não se considera invenção nem modelo de utilidade:')

In [12]:
input_docs[1]

Document(metadata={'source': 'dados/lei9279.txt'}, page_content='Art. 88. A invenção e o modelo de utilidade pertencem exclusivamente ao empregador quando decorrerem de contrato de trabalho cuja execução ocorra no Brasil e que tenha por objeto a pesquisa ou a atividade inventiva, ou resulte esta da natureza dos serviços para os quais foi o empregado contratado.      (Regulamento)\n\n        § 1º Salvo expressa disposição contratual em contrário, a retribuição pelo trabalho a que se refere este artigo limita-se ao salário ajustado.')

In [13]:
input_docs[2]

Document(metadata={'source': 'dados/lei9279.txt'}, page_content='VIII - técnicas e métodos operatórios ou cirúrgicos, bem como métodos terapêuticos ou de diagnóstico, para aplicação no corpo humano ou animal; e\n\n        IX - o todo ou parte de seres vivos naturais e materiais biológicos encontrados na natureza, ou ainda que dela isolados, inclusive o genoma ou germoplasma de qualquer ser vivo natural e os processos biológicos naturais.\n#\n        Art. 11. A invenção e o modelo de utilidade são considerados novos quando não compreendidos no estado da técnica.')

In [14]:
input_docs[3]

Document(metadata={'source': 'dados/lei9279.txt'}, page_content='Art. 22. O pedido de patente de invenção terá de se referir a uma única invenção ou a um grupo de invenções inter-relacionadas de maneira a compreenderem um único conceito inventivo.\n#\n        Art. 23. O pedido de patente de modelo de utilidade terá de se referir a um único modelo principal, que poderá incluir uma pluralidade de elementos distintos, adicionais ou variantes construtivas ou configurativas, desde que mantida a unidade técnico-funcional e corporal do objeto.\n#\n        Art. 24. O relatório deverá descrever clara e suficientemente o objeto, de modo a possibilitar sua realização por técnico no assunto e indicar, quando for o caso, a melhor forma de execução.')

In [15]:
len(input_docs)

4

In [16]:
combined_content = "\n".join(doc.page_content for doc in input_docs)
print(combined_content)


Art. 8º É patenteável a invenção que atenda aos requisitos de novidade, atividade inventiva e aplicação industrial.
#
        Art. 9º É patenteável como modelo de utilidade o objeto de uso prático, ou parte deste, suscetível de aplicação industrial, que apresente nova forma ou disposição, envolvendo ato inventivo, que resulte em melhoria funcional no seu uso ou em sua fabricação.
#
        Art. 10. Não se considera invenção nem modelo de utilidade:
Art. 88. A invenção e o modelo de utilidade pertencem exclusivamente ao empregador quando decorrerem de contrato de trabalho cuja execução ocorra no Brasil e que tenha por objeto a pesquisa ou a atividade inventiva, ou resulte esta da natureza dos serviços para os quais foi o empregado contratado.      (Regulamento)

        § 1º Salvo expressa disposição contratual em contrário, a retribuição pelo trabalho a que se refere este artigo limita-se ao salário ajustado.
VIII - técnicas e métodos operatórios ou cirúrgicos, bem como métodos terapêu

In [17]:
for doc in input_docs[:-1]:
    print(doc.metadata)
    print("\n\n" + doc.page_content)
    print("\n\n" + "-" * 30 + "\n\n")

{'source': 'dados/lei9279.txt'}


Art. 8º É patenteável a invenção que atenda aos requisitos de novidade, atividade inventiva e aplicação industrial.
#
        Art. 9º É patenteável como modelo de utilidade o objeto de uso prático, ou parte deste, suscetível de aplicação industrial, que apresente nova forma ou disposição, envolvendo ato inventivo, que resulte em melhoria funcional no seu uso ou em sua fabricação.
#
        Art. 10. Não se considera invenção nem modelo de utilidade:


------------------------------


{'source': 'dados/lei9279.txt'}


Art. 88. A invenção e o modelo de utilidade pertencem exclusivamente ao empregador quando decorrerem de contrato de trabalho cuja execução ocorra no Brasil e que tenha por objeto a pesquisa ou a atividade inventiva, ou resulte esta da natureza dos serviços para os quais foi o empregado contratado.      (Regulamento)

        § 1º Salvo expressa disposição contratual em contrário, a retribuição pelo trabalho a que se refere este artigo limit

In [18]:
answer = input_docs[0].page_content
print(answer)

Art. 8º É patenteável a invenção que atenda aos requisitos de novidade, atividade inventiva e aplicação industrial.
#
        Art. 9º É patenteável como modelo de utilidade o objeto de uso prático, ou parte deste, suscetível de aplicação industrial, que apresente nova forma ou disposição, envolvendo ato inventivo, que resulte em melhoria funcional no seu uso ou em sua fabricação.
#
        Art. 10. Não se considera invenção nem modelo de utilidade:


In [19]:
citations = input_docs[0].metadata['source']
print(citations)

dados/lei9279.txt


In [20]:
# Create Cohere's reranker with the vector DB using Cohere's embeddings as the base retriever
cohere_chat_model = ChatCohere(cohere_api_key=os.getenv("COHERE_API_KEY"))

cohere_rerank = CohereRerank(cohere_api_key=os.getenv("COHERE_API_KEY"),model="rerank-english-v3.0")

compression_retriever = ContextualCompressionRetriever(
    base_compressor=cohere_rerank, 
    base_retriever=db.as_retriever()
)

In [21]:
compressed_docs = compression_retriever.get_relevant_documents(user_query)
print(compressed_docs)

[Document(metadata={'source': 'dados/lei9279.txt', 'relevance_score': 0.99743915}, page_content='Art. 8º É patenteável a invenção que atenda aos requisitos de novidade, atividade inventiva e aplicação industrial.\n#\n        Art. 9º É patenteável como modelo de utilidade o objeto de uso prático, ou parte deste, suscetível de aplicação industrial, que apresente nova forma ou disposição, envolvendo ato inventivo, que resulte em melhoria funcional no seu uso ou em sua fabricação.\n#\n        Art. 10. Não se considera invenção nem modelo de utilidade:'), Document(metadata={'source': 'dados/lei9279.txt', 'relevance_score': 0.9929061}, page_content='Art. 88. A invenção e o modelo de utilidade pertencem exclusivamente ao empregador quando decorrerem de contrato de trabalho cuja execução ocorra no Brasil e que tenha por objeto a pesquisa ou a atividade inventiva, ou resulte esta da natureza dos serviços para os quais foi o empregado contratado.      (Regulamento)\n\n        § 1º Salvo expressa

In [22]:
user_query = "O que é um modelo de utilidade ?"

rag = CohereRagRetriever(llm=cohere_chat_model)
docs = rag.get_relevant_documents(
    user_query,
)
print(docs)

[Document(metadata={'id': 'web-search_2', 'snippet': 'Patente de Invenção / Modelo de Utilidade https://www.inovacao.ufscar.br/pt-br/propriedade-intelectual/propriedade-industrial/patente-de-invencao-modelo-de-utilidade https://www.inovacao.ufscar.br/@@site-logo/logo.svg\n\nPatente de Invenção / Modelo de Utilidade\n\nA patente é um título de propriedade concedido pelo Governo e expedido pelo Instituto Nacional de Propriedade Industrial - INPI que, por força de lei, concede direitos exclusivos de exploração e utilização de um produto, dentro dos limites do território nacional, por um período de tempo determinado.\n\nUma patente (ou privilégio) de invenção é definida como um bem material (processo, produto ou aparelho) que seja fruto da atividade intelectual do homem, e que proporcione uma melhoria no estado da técnica. Já o modelo de utilidade faz referência a um bem material já conhecido que, devido a sua forma particular, proporciona um aumento de sua capacidade de utilização (melhor

In [30]:
# https://docs.cohere.com/docs/embed-on-langchain

docs = rag.invoke(
    user_query,
    documents=input_docs,
)
# Print the documents
print("Documents:")
for doc in docs[:-1]: # resposta
    print(doc.metadata)
    print("\n\n" + doc.page_content)
    print("\n\n" + "-" * 30 + "\n\n")
# Print the final generation 
answer = docs[-1].page_content
print("Answer:")
print(answer)
# Print the final citations 
citations = docs[-1].metadata['citations']
print("Citations:")
print(citations)


Documents:
{'id': 'doc-0', 'text': 'Art. 8º É patenteável a invenção que atenda aos requisitos de novidade, atividade inventiva e aplicação industrial.\n#\n        Art. 9º É patenteável como modelo de utilidade o objeto de uso prático, ou parte deste, suscetível de aplicação industrial, que apresente nova forma ou disposição, envolvendo ato inventivo, que resulte em melhoria funcional no seu uso ou em sua fabricação.\n#\n        Art. 10. Não se considera invenção nem modelo de utilidade:'}


Art. 8º É patenteável a invenção que atenda aos requisitos de novidade, atividade inventiva e aplicação industrial.
#
        Art. 9º É patenteável como modelo de utilidade o objeto de uso prático, ou parte deste, suscetível de aplicação industrial, que apresente nova forma ou disposição, envolvendo ato inventivo, que resulte em melhoria funcional no seu uso ou em sua fabricação.
#
        Art. 10. Não se considera invenção nem modelo de utilidade:


------------------------------


Answer:
Um mode

In [28]:
for doc in docs:
    print(doc.metadata)
    print("\n\n" + "+" * 30 + "\n\n")
    print("\n\n" + doc.page_content)
    print("\n\n" + "-" * 30 + "\n\n")

{'id': 'doc-0', 'text': 'Art. 8º É patenteável a invenção que atenda aos requisitos de novidade, atividade inventiva e aplicação industrial.\n#\n        Art. 9º É patenteável como modelo de utilidade o objeto de uso prático, ou parte deste, suscetível de aplicação industrial, que apresente nova forma ou disposição, envolvendo ato inventivo, que resulte em melhoria funcional no seu uso ou em sua fabricação.\n#\n        Art. 10. Não se considera invenção nem modelo de utilidade:'}


++++++++++++++++++++++++++++++




Art. 8º É patenteável a invenção que atenda aos requisitos de novidade, atividade inventiva e aplicação industrial.
#
        Art. 9º É patenteável como modelo de utilidade o objeto de uso prático, ou parte deste, suscetível de aplicação industrial, que apresente nova forma ou disposição, envolvendo ato inventivo, que resulte em melhoria funcional no seu uso ou em sua fabricação.
#
        Art. 10. Não se considera invenção nem modelo de utilidade:


------------------------

In [27]:
len(docs)

2